In [43]:
import pandas as pd
import numpy as np
import os

DATA_PATH ="../train/"
# 1. 데이터 로드
# train_ratings는 csv, genres는 tsv 파일입니다.

train_ratings = pd.read_csv(os.path.join(DATA_PATH,'train_ratings.csv'))
df_genres = pd.read_csv(os.path.join(DATA_PATH,'genres.tsv'),sep='\t')
train_ratings.head(5)
df_genres.head(5)
# 2. 데이터 병합 (Merge)
# train_ratings를 기준으로 df_genres를 붙입니다.
# TODO: on에는 공통 컬럼명, how에는 병합 방식을 적어주세요.
df_merged = pd.merge(train_ratings, df_genres, on='item', how='left')

# 3. 결측치 처리 (Fillna)
# 장르가 없는 경우(NaN) 튕겨내지 말고 'Unknown'으로 채워줍니다.
df_merged['genre'] = df_merged['genre'].fillna('Unknown')

# 검증
print(f"Total Log Count: {len(df_merged)}")
print(df_merged.head())

Total Log Count: 14126324
   user  item        time      genre
0    11  4643  1230782529     Action
1    11  4643  1230782529  Adventure
2    11  4643  1230782529      Drama
3    11  4643  1230782529     Sci-Fi
4    11   170  1230782534     Action


In [ ]:
## 실험 설계 무엇을 input label 로 할지 나누는 작업

## 검증 전략 각 유저가 마지막으로 본 아이템 딱 1개를 정답으로 숨겨두고  나머지 과거 데이터만 가지고 맞출 수 있는지
## validate로 구성하는 방법
df_sorted=df_merged.sort_values(['user','time'])

df_sorted.head()
## duplicated로 각 중복 user id중 가장 마지막을 추출함
df_test = df_sorted.drop_duplicates(subset=['user'], keep='last')
# 가장 마지막 인덱스를 통해서 드롭함
df_temp = df_sorted.drop(df_test.index)
df_val = df_temp.drop_duplicates(subset="user",keep='last')

df_train = df_temp.drop(df_val.index)
# 검증
print(f"Train: {len(df_train)}, Valid: {len(df_val)}, Test: {len(df_test)}")

Train: 14063604, Valid: 31360, Test: 31360


In [45]:
from collections import Counter
import itertools as it  # (A) 라이브러리 이름
# 윈도우를 5개로 정하나요?
def train_cooc_dict(df, window_size=5):
    # counter 이름을 왜 cooc라고 지은거지
    cooc_counts = Counter()
    
    # user로 묶는데 item을 []로 감싸면 무슨효과로 item이 시간순 리스트가 되는거죠
    user_item_list = df.groupby('user')['item'].apply(list)
    
    for items in user_item_list:
    
        # 아이템 리스트가 [A,B,C,D]
        for i in range(len(items)):
            # 현재 타깃 i에 대해서 윈도우 만큼의 애들을 자름 파이썬이라 index 오류는 안남..
            context_items = items[i+i : i+1+window_size]
            
            # 윈도우에 있는 context_items에서 이웃을 짝짔는다
            for neigbor in context_items:
                # (A) -> (B) 방향
                cooc_counts[(items[i], neigbor)] += 1
                # (B) -> (A) 방향
                cooc_counts[(neigbor,items[i])]+= 1
    return cooc_counts
cooc_dict = train_cooc_dict(df_train, window_size=5)
print(f"Top 5 Pairs: {cooc_dict.most_common(5)}")

Top 5 Pairs: [((5445, 5445), 3116), ((4306, 4306), 2968), ((8961, 8961), 2736), ((1215, 1215), 2362), ((29, 29), 2296)]


In [46]:
from collections import defaultdict

def cooc_to_map(cooc_counts):
    # item_id를 키로 하고 값은 Counter인 딕셔너리?
    item_cooc_map = defaultdict(Counter)
    
    for (item_i, item_j), count in cooc_counts.items():
        # 두가지 아이템조합의 dict -> 2차원 배열로 만들어서 관리한다
        #그래서 두가지 아이템이 있으면 그 카운트를 뱉는 맵을 생성하는거군요
        item_cooc_map[item_i][item_j] = count
        
    return item_cooc_map
item_map = cooc_to_map(cooc_dict)

test_item = list(item_map.keys())[0]
print(type(item_map.keys()))
print(f"item{test_item}의 연관 아이템 Top3:")
#mostcommon이 어떻게 쓰이는지도 까먹은 것 같네요
print(item_map[test_item].most_common(3))

<class 'dict_keys'>
item4643의 연관 아이템 Top3:
[(4643, 246), (1882, 21), (6537, 18)]


In [47]:
# 1. 유저별 과거 기록 가져오기 (Retrieval의 힌트로 사용)
# train_df는 이미 시간순 정렬이 되어 있습니다.
user_history_dict = df_train.groupby('user')['item'].apply(list).to_dict()

global_popular = df_train['item'].value_counts().head(50).index.tolist()

def generate_candidates_robust(user_id, history_dict, cooc_map, n_candidates=50):
    candidates = []
    
    # 1. Retrieval 시도 (History & Co-occurrence)
    if user_id in history_dict:
        last_item = history_dict[user_id][-1]
        
        if last_item in cooc_map:
            # 짝꿍 아이템 가져오기
            top_items = cooc_map[last_item].most_common(n_candidates)
            candidates = [item for item, score in top_items]
    
    # 2. [수정된 부분] 모자란 개수만큼 인기 아이템으로 채우기 (Fallback)
    if len(candidates) < n_candidates:
        for item in global_popular:
            if item not in candidates: # 중복 방지
                candidates.append(item)
                # 개수가 다 차면 중단
                if len(candidates) >= n_candidates:
                    break
                    
    return candidates

# 검증: Valid Set의 첫 번째 유저에 대해 테스트
test_user = df_val['user'].iloc[0]
candidates = generate_candidates_robust(test_user, user_history_dict, item_map)

print(f"User {test_user}의 Last Item: {user_history_dict[test_user][-1]}")
print(f"추천된 후보군 개수: {len(candidates)}")
print(f"후보군 Top 10: {candidates[:10]}")

User 11의 Last Item: 7153
추천된 후보군 개수: 50
후보군 Top 10: [7153, 2571, 4993, 5952, 318, 5445, 527, 4034, 1500, 58559]


In [48]:
# ---------------------------------------------------------
# Step 4. Candidate Generation & Evaluation (Recall@50)
# ---------------------------------------------------------

# 1. 빠른 조회를 위한 유저 히스토리 (Train Set 기준)
# 검색 속도를 위해 DataFrame -> Dictionary 변환
train_history = df_train.groupby('user')['item'].apply(list).to_dict()

def evaluate_retrieval_strict(val_data, cooc_map, history_dict, top_k=50):
    hits = 0
    total = 0
    
    for _, row in val_data.iterrows():
        user = row['user']
        truth = row['item']
        
        # [수정 1] Fallback이 적용된 함수를 사용해 무조건 후보를 받아옴
        recommendations = generate_candidates_robust(user, history_dict, cooc_map, n_candidates=top_k)
            
        # [수정 2] continue 없이 무조건 채점
        if truth in recommendations:
            hits += 1
        
        total += 1 # 분모는 무조건 증가
    
    return hits / total if total > 0 else 0

# 다시 측정해보면 점수가 살짝 떨어질 수 있습니다 (하지만 이게 진짜 실력입니다)
real_recall = evaluate_retrieval_strict(df_val, item_map, train_history, top_k=50)
print(f"Real Retrieval Recall@50: {real_recall:.4f}")

print(f"Retrieval Recall@10: {real_recall:.4f}")

Real Retrieval Recall@50: 0.5991
Retrieval Recall@10: 0.5991


In [49]:
def make_ranking_dataset(val_data, cooc_map, history_dict, top_k=50):
    ranking_data = []
    
    # tqdm이 있다면: for _, row in tqdm(val_data.iterrows()):
    for _, row in val_data.iterrows():
        user = row['user']
        truth = row['item']
        
        last_item = history_dict[user][-1]
        
        if last_item in cooc_map:
            candidates_with_score = cooc_map[last_item].most_common(top_k)
        else:
            continue
            
        for cand_item, score in candidates_with_score:
            label = 1 if truth == cand_item else 0
            
            ranking_data.append({
                'user': user,
                'item': cand_item,
                'cooc_score': score,
                'label': label
            })
    
    # [Fix] return은 for문이 완전히 끝난 뒤에 실행되어야 합니다. (들여쓰기 주의!)
    return pd.DataFrame(ranking_data)

# 다시 실행
df_rank_train = make_ranking_dataset(df_val, item_map, train_history, top_k=50)

# 검증
print(f"Ranking Train Size: {len(df_rank_train)}")
print(df_rank_train['label'].value_counts())

Ranking Train Size: 1127086
label
0    1108310
1      18776
Name: count, dtype: int64


In [50]:
## Item Meta 생성
#Merge : df_rank_train 에 장르 정보를 붙인다
## 아이템별 대표 장르 1개 추출
df_item_features = df_genres.drop_duplicates(subset=['item'],keep='first')

# 2. Ranking 데이터에 장르 붙이기 
# User, Item,Score, Label + df_item_features(Item, Genre)
df_final_train = pd.merge(df_rank_train, df_item_features, on='item',how='left')

df_final_train['genre'] = df_final_train['genre'].fillna('Unknown')

print(df_final_train.head())

   user  item  cooc_score  label      genre
0    11  7153        1880      0     Action
1    11  2571         392      0     Action
2    11  4993         357      0  Adventure
3    11  5952         319      0  Adventure
4    11   318         199      0      Crime


In [51]:
# ---------------------------------------------------------
# Step 7. Train Ranking Model (CatBoost)
# ---------------------------------------------------------
from catboost import CatBoostClassifier

# 1. 학습에 사용할 Feature(문제)와 Label(정답) 정의
# 우리는 '점수'와 '장르' 두 가지 힌트만 줍니다. (심플 이즈 베스트)
features = ['cooc_score', 'genre']
X = df_final_train[features]
y = df_final_train['label']

# 2. CatBoost 모델 선언
# task_type="GPU"를 쓰면 RTX 3070을 쓸 수 있지만, 설정이 까다로우니 일단 CPU로 돌립니다. (충분히 빠릅니다)
model = CatBoostClassifier(
    iterations=500,       # 반복 학습 횟수
    learning_rate=0.1,    # 학습 속도
    depth=6,              # 의사결정나무의 깊이
    cat_features=['genre'], # [핵심] "장르는 글자니까 네가 알아서 처리해"
    verbose=100           # 100번마다 로그 출력
)

# 3. 학습 시작 (Fit)
print("Training CatBoost Ranker...")
model.fit(X, y)

# 4. 중요도 확인 (어떤 힌트가 정답을 맞추는 데 중요했을까?)
print("\nFeature Importance:")
for name, imp in zip(features, model.get_feature_importance()):
    print(f"{name}: {imp:.2f}")

Training CatBoost Ranker...
0:	learn: 0.4846665	total: 287ms	remaining: 2m 23s
100:	learn: 0.0531592	total: 28.5s	remaining: 1m 52s
200:	learn: 0.0500265	total: 58.6s	remaining: 1m 27s
300:	learn: 0.0480232	total: 1m 29s	remaining: 59.5s
400:	learn: 0.0467933	total: 1m 58s	remaining: 29.3s
499:	learn: 0.0460221	total: 2m 26s	remaining: 0us

Feature Importance:
cooc_score: 94.26
genre: 5.74


In [52]:
def evaluate_reranker(test_data, model, cooc_map, history_dict, item_meta, top_k=50):
    hits = 0 
    total = 0
    
    # Test Set의 모든 유저에 대해 반복
    for _, row in test_data.iterrows():
        user = row['user']
        truth = row['item']
        
        if user not in history_dict: continue
        last_item = history_dict[user][-1]
        
        if last_item not in cooc_map:
            recommendations = []
        else:
            # 후보 50개와 점수
            candidates = cooc_map[last_item].most_common(top_k)
            
            #2. Reranking을 위한 데이터 프레임
            # 모델에게 이 50개 채점해주라는 표를 만든다
            inf_df = pd.DataFrame(candidates, columns=['item', 'cooc_score'])
            # 장르 정보 붙이기
            inf_df = pd.merge(inf_df, item_meta, on='item', how='left')
            inf_df['genre'] = inf_df['genre'].fillna('Unknown')
            # 3. CatBoost 예측
            # 정답일 확률을 물어보는 함수?
            # [:, 1] 클래스 1(정답일 확률을 가져온다??)
            scores = model.predict_proba(inf_df[['cooc_score','genre']])[:,1]
            
            # 4. 점수순으로 다시 정렬 (Reranking)
            inf_df['pred_score'] = scores
            inf_df = inf_df.sort_values('pred_score', ascending=False)
            
            # 최종 추천 리스트 (아이템 ID만 추출)
            recommendations = inf_df['item'].tolist()
        
        if truth in recommendations[:top_k]:
            hits += 1 
        total += 1
    return hits/ total if total > 0 else 0
print("Evaluating Final Reranker on Test Set...")
final_recall = evaluate_reranker(df_test, model, item_map, train_history, df_item_features, top_k=50)

print(f"Final Test Recall@50: {final_recall:.4f}")

Evaluating Final Reranker on Test Set...
Final Test Recall@50: 0.5072


In [ ]:
# ---------------------------------------------------------
# Robust Evaluation: 통계 왜곡 방지 버전
# ---------------------------------------------------------

# 0. 비상용 인기 아이템 50개 미리 준비 (Train 기준)
global_popular = df_train['item'].value_counts().head(50).index.tolist()

def evaluate_robust(val_data, model, cooc_map, history_dict, item_meta, cand_n=200, top_k=10):
    hits = 0
    total = 0
    
    # 전체 유저에 대해 예외 없이 반복
    for _, row in val_data.iterrows():
        user = row['user']
        truth = row['item']
        
        recommendations = []
        
        # [Case 1] 기록이 있는 유저 (정상적인 개인화 추천)
        if user in history_dict:
            last_item = history_dict[user][-1]
            
            if last_item in cooc_map:
                # 1. Retrieval
                candidates = cooc_map[last_item].most_common(cand_n)
                
                # 2. Reranking Setup
                inf_df = pd.DataFrame(candidates, columns=['item', 'cooc_score'])
                inf_df = pd.merge(inf_df, item_meta, on='item', how='left')
                inf_df['genre'] = inf_df['genre'].fillna('Unknown')
                
                # 3. CatBoost Scoring
                scores = model.predict_proba(inf_df[['cooc_score', 'genre']])[:, 1]
                inf_df['pred_score'] = scores
                inf_df = inf_df.sort_values('pred_score', ascending=False)
                
                recommendations = inf_df['item'].tolist()

        # [Case 2] 추천 개수가 모자라거나, 기록이 없는 경우 (Fallback)
        # 인기 아이템으로 빈자리를 채웁니다.
        if len(recommendations) < top_k:
            for item in global_popular:
                if item not in recommendations:
                    recommendations.append(item)
                    if len(recommendations) >= top_k:
                        break
        
        # 상위 k개 자르기
        final_recs = recommendations[:top_k]
        
        # [핵심] continue 없이 무조건 채점 (Hits Check)
        if truth in final_recs:
            hits += 1
            
        total += 1  # 분모(Total)는 무조건 증가
        
    return hits / total if total > 0 else 0

# 실행: 진짜 실력 확인
print("Checking Robust Recall@10...")
real_recall = evaluate_robust(df_test, model, item_map, train_history, df_item_features, cand_n=50, top_k=10)

print(f"✅ Real Recall@10: {real_recall:.4f}")

Checking Robust Recall@10...
✅ Real Recall@10: 0.4869


: 

In [56]:
# ---------------------------------------------------------
# Step 9. Generate Final Submission (Correction)
# ---------------------------------------------------------
from tqdm import tqdm

# 0. 비상용 인기 아이템 50개 미리 준비 (Train 데이터 기준)
# (이건 추천할 게 없을 때를 대비한 '비상약'입니다)
global_popular = df_train['item'].value_counts().head(50).index.tolist()

def generate_submission(target_users, model, cooc_map, history_dict, item_meta, cand_n=200, top_k=10):
    results = []
    
    print(f"Generating Predictions for {len(target_users)} Users...")
    
    # tqdm으로 진행 상황 확인
    for user in tqdm(target_users):
        recommendations = []
        
        # --- A. 개인화 추천 시도 (Retrieval + Reranking) ---
        # 1. 유저의 과거 기록(Train)이 있는지 확인
        if user in history_dict:
            last_item = history_dict[user][-1]
            
            # 2. Retrieval: 짝꿍 아이템이 있는지 확인
            if last_item in cooc_map:
                candidates = cooc_map[last_item].most_common(cand_n)
                
                # 3. Reranking Data Setup
                inf_df = pd.DataFrame(candidates, columns=['item', 'cooc_score'])
                inf_df = pd.merge(inf_df, item_meta, on='item', how='left')
                inf_df['genre'] = inf_df['genre'].fillna('Unknown')
                
                # 4. CatBoost Scoring (확률 계산)
                scores = model.predict_proba(inf_df[['cooc_score', 'genre']])[:, 1]
                
                # 5. Sort (점수 높은 순 정렬)
                inf_df['pred_score'] = scores
                inf_df = inf_df.sort_values('pred_score', ascending=False)
                recommendations = inf_df['item'].tolist()
        
        # --- B. 빈칸 채우기 (Fallback) ---
        # 추천이 아예 없거나(Cold Start), 10개보다 적은 경우 인기 아이템으로 채움
        if len(recommendations) < top_k:
            for item in global_popular:
                if item not in recommendations: # 중복 방지
                    recommendations.append(item)
                    if len(recommendations) >= top_k:
                        break
        
        # 최종 Top 10 자르기
        final_recs = recommendations[:top_k]
        
        # 결과 저장
        for item in final_recs:
            results.append({
                'user': user,
                'item': item
            })
            
    return pd.DataFrame(results)

# ---------------------------------------------------------
# 실행 파트 (수정됨)
# ---------------------------------------------------------

# 1. [핵심 수정] 외부 파일 대신, 우리 Test Set에서 유저 리스트를 직접 추출합니다.
# df_test에는 모든 유저가 1줄씩 들어있으므로, 유저 ID 리스트와 같습니다.
target_user_list = df_test['user'].unique()

print(f"Total Target Users: {len(target_user_list)}명")

# 2. 함수 실행
df_final = generate_submission(target_user_list, model, item_map, train_history, df_item_features)

# 3. CSV 저장
df_final.to_csv('my_final_submission.csv', index=False)

# 4. 검증 (유저 수 x 10 = 313,600 줄이 나와야 성공!)
print(f"\nFinal Shape: {df_final.shape}")
print(df_final.head(10))

Total Target Users: 31360명
Generating Predictions for 31360 Users...


100%|██████████| 31360/31360 [01:16<00:00, 412.54it/s]



Final Shape: (313600, 2)
   user  item
0    11  7153
1    11  5952
2    11  8961
3    11  1198
4    11  1639
5    11   260
6    11  4886
7    11  2571
8    11  4993
9    11  2268


In [55]:
# ---------------------------------------------------------
# Step 10. Final Logic Fix (Full History Update)
# ---------------------------------------------------------

# 1. 모든 데이터를 하나로 합칩니다. (가장 최신 기록을 찾기 위해)
# df_train, df_val, df_test를 모두 합쳐야 '진짜 마지막 아이템'을 알 수 있습니다.
df_all = pd.concat([df_train, df_val, df_test])

# 2. 전체 데이터 기준으로 최신 History Dict를 다시 만듭니다.
# 이제 user_id를 넣으면, Train 기간이 아니라 '데이터 전체 기간'의 맨 마지막 아이템이 나옵니다.
final_history_dict = df_all.groupby('user')['item'].apply(list).to_dict()

# 3. (선택사항) Co-occurrence Map도 전체 데이터로 다시 만들면 더 좋습니다.
# 시간이 걸린다면 기존 map을 써도 되지만, 성능을 위해선 업데이트를 권장합니다.
# print("Re-training Co-occurrence Map with Full Data...")
# final_cooc_map = train_cooc_map(df_all, window_size=5)
# 일단은 기존 map을 그대로 써도 점수는 확 오를 겁니다.
final_cooc_map = item_map 

# ---------------------------------------------------------
# 다시 제출 파일 생성
# ---------------------------------------------------------

# 4. 업데이트된 final_history_dict를 넣고 다시 실행
print("Generating Real Submission with Full History...")

df_final_submission = generate_submission(
    target_user_list,    # 아까 만든 전체 유저 리스트
    model,               # 학습된 CatBoost 모델
    final_cooc_map,      # (선택) 전체 데이터로 업데이트된 맵 or 기존 맵
    final_history_dict,  # [핵심] Train+Val+Test가 모두 합쳐진 최신 기록
    df_item_features,    # 아이템 메타 정보
    cand_n=50, 
    top_k=10
)

# 5. 저장
df_final_submission.to_csv('submission_fixed.csv', index=False)
print(f"Fixed Submission Saved! Shape: {df_final_submission.shape}")

Generating Real Submission with Full History...
Generating Predictions for 31360 Users...


100%|██████████| 31360/31360 [01:08<00:00, 457.77it/s]


Fixed Submission Saved! Shape: (313600, 2)
